In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

import xml.etree.ElementTree as ET
# import requests
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## B. Importing Pieces

### B.1 Import a Single Piece

In [2]:
# only needed to update metadata in MEI 4.0

MEINSURI = "http://www.music-encoding.org/ns/mei"
XMLNS = "http://www.w3.org/XML/1998/namespace"

# register MEINS as the default namespace for a cleaner serialization
ET.register_namespace('', MEINSURI)

# path = 'Music_Files/test.mei'
path = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0001.mei'
if "http" in path:
    mei_doc = ET.fromstring(requests.get(path).text)
else:
    with open(path, "r") as file:
        mei_doc = ET.fromstring(file.read())
        
if mei_doc.get('meiversion').startswith('4.'):
    piece = importScore(path)
    # mei_doc = ET.fromstring(requests.get(path).text)
    title = mei_doc.find('mei:meiHead//mei:titleStmt/mei:title', namespaces={"mei": MEINSURI}).text
    composer = mei_doc.find('mei:meiHead//mei:titleStmt//mei:persName[@role="composer"]', namespaces={"mei": MEINSURI}).text
    piece.score.metadata.title = title
    piece.score.metadata.composer = composer
    print("mei 4 file")
    print(piece.score.metadata.title)
    print(piece.score.metadata.composer)

else:
    piece = importScore(path)
    print("mei 3 file")
    print(piece.score.metadata.title)
    print(piece.score.metadata.composer)


Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0001.mei
mei 4 file
Veni speciosam
Lupi, Johannes


In [10]:
# THIS DOES NOT WORK
# MEINSURI = "http://www.music-encoding.org/ns/mei"
# XMLNS = "http://www.w3.org/XML/1998/namespace"

# # register MEINS as the default namespace for a cleaner serialization
# ET.register_namespace('', MEINSURI)

# path = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0008.mei'
# mei_doc = ET.fromstring(requests.get(path).text)

# find all supplied elements
supplieds = mei_doc.findall('.//mei:supplied', namespaces={"mei": MEINSURI})
# find all their parents (this is not a typical XML processing pattern, but ET doesn't offer a "get parent" method)
parents = mei_doc.findall('.//mei:supplied/..', namespaces={"mei": MEINSURI})

# loop through the supplied elements keeping track of index
for [i, supplied] in enumerate(supplieds):
    # store children (the accid)
    children = supplied.getchildren()
    # get the right parent fomr the parent elements list
    parent = parents[i]
    # figure out where the supplied is among its siblings (is it the first child, second? etc.)
    supplied_position = list(parent).index(supplied)
    # remove the supplied 👋
    parent.remove(supplied)
    # place supplied's children at the same location as the supplied.
    # loop through the children in case there's more than one (shouldn't happen, but just in case)
    for [j, child] in enumerate(children):
        parent.insert(supplied_position + j, child)

# mei_doc is now altered and can be passed on safely to music21.

# convert mei_doc to string before passing to CRIM Intervals
meidata = ET.tostring(mei_doc)

/var/folders/k5/k4j_550s2yq5776vgmw6r_xr0000gp/T/ipykernel_69340/1175575174.py:18: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  children = supplied.getchildren()
